In [12]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# 연습문제 :네이버 영화 리뷰 모으기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import time
import sys
import math
import numpy  
import pandas as pd 
import os
import random

print("=" *80)
print("연습문제 :네이버 영화 리뷰 정보 수집하기 ")
print("=" *80)

query_txt = input('1.크롤링 할 영화의 제목을 입력하세요: ')
query_url = 'https://movie.naver.com'

cnt = int(input('2.크롤링 할 리뷰건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)

f_dir = input("3.파일을 저장할 폴더명만 쓰세요(예:c:\\py_temp\\):")
if f_dir =='' :
    f_dir='c:\\py_temp\\'
    
print("-" *80)
print("요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~~~")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
s_time = time.time( )

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

driver.get(query_url)
driver.maximize_window()
time.sleep(2)

# 영화 제목으로 검색
element = driver.find_element(By.ID,"ipt_tx_srch")
element.send_keys(query_txt)
driver.find_element(By.XPATH,'//*[@id="jSearchArea"]/div/button').click()
driver.find_element(By.XPATH,'//*[@id="old_content"]/ul[2]/li/dl/dt/a').click()
        
#Step 4 평점 옵션 버튼 클릭
driver.find_element(By.LINK_TEXT,"평점").click()

#Step 5. 현재 페이지 리뷰와 점수 등 내용 수집
# IFRAME 변경
driver.switch_to.frame('pointAfterListIframe') #Iframe 구조로 되어있기 때문에 iframe name으로 pointAfterListIframe driver을 바꿔준 다음 진행한다.


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#현재 총 리뷰 건수를 확인하여 사용자의 요청건수와 비교 후 동기화
result= soup.find('div', class_='score_total').find('strong','total').find('em').get_text()



In [ ]:
print("=" *80)
search_cnt = int(result.replace(",",""))

if cnt > search_cnt :
    cnt = search_cnt

print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)

print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)


score2=[]
review2=[]
writer2=[]
wdate2=[]
gogam2=[]
g_gogam2=[]
b_gogam2=[]
dwlist2=[]

count = 0         # 크롤링 건수 카운트 변수
click_count = 1   # 페이지 번호

while ( click_count <= page_cnt )  :
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            score_result = soup.find('div', class_='score_result').find('ul')
            slist = score_result.find_all('li')



            for li in slist:

                count += 1
                
                f = open(ff_name, 'a',encoding='UTF-8')
            
                print("\n")
                print("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다==============" %(cnt,count))
                score = li.find('div', class_='star_score').find('em').get_text()
                print("1.별점:","*"*int(score),": ", score)
                score2.append(score)
                f.write("\n")
                f.write("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다==============" %(cnt,count) + "\n")
                f.write("1.별점:"+score + "\n")
            
                review = li.find('div', class_='score_reple').find('p').get_text().replace('\n','').strip()
                print("2.리뷰내용:",review)
                f.write("2.리뷰내용:" + review + "\n")
                review2.append(review)
         
                dwlist = li.find('div',class_='score_reple').find_all('em')
                writer = dwlist[0].find('span').get_text()
                print("3.작성자:",writer)
                f.write("3.작성자:" + writer + "\n")
                writer2.append(writer)
                
                wdate = dwlist[1].text
                print('4.작성일자:',wdate)
                f.write("4.작성일자:" + wdate + "\n")
                wdate2.append(wdate)
            
                gogam = li.find('div', class_='btn_area').find_all('strong')
                g_gogam = gogam[0].text
                print('5.공감:',g_gogam)
                f.write("5.공감:" + g_gogam + "\n")
                g_gogam2.append(g_gogam)
                
                b_gogam = gogam[1].text
                print('6.비공감:', b_gogam)
                f.write("6.비공감:" + b_gogam + "\n")
                b_gogam2.append(b_gogam)
                
                if count == cnt :
                    break
            
            time.sleep(random.randrange(1,2))  

            click_count += 1
            
            if click_count > page_cnt :
                break
            else :
                driver.find_element(By.LINK_TEXT,"%s" %click_count).click()

            time.sleep(random.randrange(1,2))  # 3-8 초 사이에 랜덤으로 시간 선택

In [15]:
review2

['관람객\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t액션이 제대로 미친듯 ㄷㄷ 김다미 임펙트 쩜',
 '관람객\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t마녀1에서 중2병감성좀 줄이면 되겠다고 그렇게 원성이 많았는데 감독이 그거에 빡돌았는지 중2감성을 곱빼기로 쑤셔넣었네요.마녀1에서부터 이어진 내안의 흑염룡 힘숨찐 후까시가 영화 80%를 차지하고있어서 진짜 가만히앉아...']

In [ ]:
#Step 11. xls 형태와 csv 형태로 저장하기
naver_movie = pd.DataFrame()
naver_movie['별점(평점)']=score2
naver_movie['리뷰내용']=review2
naver_movie['작성자']=writer2
naver_movie['작성일자']=wdate2
naver_movie['공감횟수']=g_gogam2
naver_movie['비공감횟수']=b_gogam2

# csv 형태로 저장하기
naver_movie.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
naver_movie.to_excel(fx_name ,index=True,engine='openpyxl')

e_time = time.time( )
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )